In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Utility to generate random BST

In [4]:
def generateBST(N: int, min_: int, max_: int) -> BinaryTreeNode|None:
    def insert(curr: BinaryTreeNode|None, n: int) -> BinaryTreeNode:
        if not curr:
            curr = BinaryTreeNode(n)
        elif curr.val < n:
            curr.right = insert(curr.right, n)
        else:
            curr.left = insert(curr.left, n)

        return curr

    assert N <= max_ - min_, "Number of available samples must be >= N"
    root: BinaryTreeNode|None = None
    for n in np.random.choice(np.arange(min_, max_), size=N, replace=False):
        root = insert(root, n)

    return root

LC Medium: Longest Substring with atleast K repeating characters

In [5]:
# https://leetcode.com/problems/longest-substring-with-at-least-k-repeating-characters/
# https://leetcode.com/problems/longest-substring-with-at-least-k-repeating-characters/submissions/1286506890
def longestSubstring(s: str, k: int) -> int:
    """
    Count the freq of all characters, split on the characters with freq < k
    """
    N = len(s)

    # Count the freq
    freq: collections.Counter = collections.Counter(s)
    is_valid = not any(map(lambda x: x < k, freq.values()))

    # If the entire string is okay, return length as answer
    if is_valid:
        return N

    # Split on the invalid characters and backtrack inside
    else:
        max_length = start_idx = 0
        for curr_idx in range(N):
            if 0 < freq[s[curr_idx]] < k:
                max_length = max(max_length, longestSubstring(s[start_idx:curr_idx], k))
                start_idx = curr_idx + 1

        max_length = max(max_length, longestSubstring(s[start_idx:], k))
        return max_length

# Tesitng the solution
assert longestSubstring("aaabb", 3) == 3
assert longestSubstring("ababbc", 2) == 5
assert longestSubstring("bbaaa", 3) == 3

LC Virtual - 4: Weekly Contest 5
https://leetcode.com/contest/leetcode-weekly-contest-5/

In [6]:
# Q4: Frog Jump
def canCrossRecursive(stones: list[int]) -> bool:
    @functools.cache
    def backtrack(n: int, jump: int) -> bool:
        if jump < 0:
            return False
        elif n == stones[-1]:
            return True
        else:
            if n + jump - 1 in positions and backtrack(n + jump - 1, jump - 1):
                return True
            elif jump > 0 and n + jump in positions and backtrack(n + jump, jump):
                return True
            elif n + jump + 1 in positions and backtrack(n + jump + 1, jump + 1):
                return True
            else:
                return False

    N = len(stones)
    positions: dict[int, int] = {n: i for i, n in enumerate(stones)}
    return len(stones) <= 1 or backtrack(stones[0], 0)

# Testing the solution
assert canCrossRecursive([0,1,3,5,6,8,12,17]) == True
assert canCrossRecursive([0,1,2,3,4,8,9,11]) == False
assert canCrossRecursive([]) == True
assert canCrossRecursive([100]) == True
assert canCrossRecursive([0,2]) == False

In [7]:
def canCrossTabulation(stones: list[int]) -> bool:
    N = len(stones)
    positions: dict[int, int] = {n: i for i, n in enumerate(stones)}
    dp: list[list[bool]] = [[False if stones[i] != stones[-1] else True for jump in range(N + 2)] for i in range(N)]
    for i in range(N - 2, -1, -1):
        for jump in range(N):
            n = stones[i]
            if n + jump - 1 in positions:
                dp[i][jump] = dp[i][jump] or dp[positions[n + jump - 1]][jump - 1]
            if jump > 0 and n + jump in positions:
                dp[i][jump] = dp[i][jump] or dp[positions[n + jump]][jump]
            if n + jump + 1 in positions:
                dp[i][jump] = dp[i][jump] or dp[positions[n + jump + 1]][jump + 1]

    return len(stones) <= 1 or dp[0][0]

# Testing the solution
assert canCrossTabulation([0,1,3,5,6,8,12,17]) == True
assert canCrossTabulation([0,1,2,3,4,8,9,11]) == False
assert canCrossTabulation([]) == True
assert canCrossTabulation([100]) == True
assert canCrossTabulation([0,2]) == False

In [8]:
# Q3: Remove K digits
def removeKdigits(num: str, k: int) -> str:
    pass

In [9]:
# Q1: Find Nth Digit
def findNthDigit(N: int) -> int:
    def countDigitsBefore(n: int) -> int:
        if n < 10:
            return n
        else:
            n_digits = math.floor(math.log10(n)) + 1
            smallest_n_digit_number = 10 ** (n_digits - 1)
            n_digits_before = (n - smallest_n_digit_number + 1) * n_digits
            return n_digits_before + countDigitsBefore(smallest_n_digit_number - 1)

    # Binary search
    low, high = 1, N
    while low <= high:
        mid = (low + high) // 2
        count = countDigitsBefore(mid)
        if count == N:
            return mid % 10
        elif count < N:
            low = mid + 1
        else:
            high = mid - 1

    delta = countDigitsBefore(low) - N
    return int(str(low)[-delta-1])

# Testing the solution
assert findNthDigit(11) == 0
assert findNthDigit(3) == 3

In [10]:
# Q2: Binary Watch
def readBinaryWatch(turnedOn: int) -> set[str]:
    watch: list[bool] = [False for i in range(10)]
    result: set[str] = set()
    def backtrack(lights: int) -> None:
        if lights == 0:
            hour = minute = 0
            for i in range(10):
                if watch[i]:
                    if i < 4:
                        hour += 2 ** (3 - i)
                    else:
                        minute += 2 ** (9 - i)

            if 0 <= hour < 12 and 0 <= minute <= 59:
                result.add(f"{hour}:{str(minute).zfill(2)}")

        else:
            for i in range(10):
                if not watch[i]:
                    watch[i] = True
                    backtrack(lights - 1)
                    watch[i] = False

    backtrack(turnedOn)
    return result

# Testing the solution
assert sorted(readBinaryWatch(1)) == sorted(["0:01","0:02","0:04","0:08","0:16","0:32","1:00","2:00","4:00","8:00"])

Minimum no of moves to seat everyone 
https://leetcode.com/problems/minimum-number-of-moves-to-seat-everyone

In [ ]:
def minMovesToSeat(seats: list[int], students: list[int]) -> int:
    # Compute length
    N = len(seats)

    # Sort both to ensure that we get the minimal distances
    seats.sort()
    students.sort()

    # Compute abs distance between seats and students
    moves = 0
    for i in range(N):
        moves += abs(seats[i] - students[i])
    return moves

# Testing the solution
assert minMovesToSeat([3,1,5], [2,7,4]) == 4
assert minMovesToSeat([2,2,6,6], [1,3,2,6]) == 4

LC Medium: Minimum increment to make array unique
https://leetcode.com/problems/minimum-increment-to-make-array-unique

In [ ]:
def minIncrementForUnique(nums: list[int]) -> int:
    # Count freq & unique values present
    counts: dict[int, int] = dict()
    unique: set[int] = set()
    for n in nums:
        counts[n] = counts.get(n, 0) + 1
        unique.add(n)

    # If count of any number is greater than 1, shift right
    moves = 0
    while unique:
        key = unique.pop()
        count = counts.pop(key)
        if count > 1:
            shift = count - 1
            moves += shift
            counts[key + 1] = counts.get(key + 1, 0) + shift
            unique.add(key + 1)

    return moves

# Testing the solution
assert minIncrementForUnique([1,2,2]) == 1
assert minIncrementForUnique([3,2,1,2,1,7]) == 6
assert minIncrementForUnique([1,2,2,5,5]) == 2

Maximum Gap
https://leetcode.com/problems/maximum-gap/

In [ ]:
# https://leetcode.com/problems/maximum-gap/submissions/1288173457/
def maximumGap(nums: list[int]) -> int:
    """
    1. Maximum gap is always greater than or equal to ceil((max - min) / N)
        - Divide N nums into N - 1 groups
        - If evenly distributed, each elements would be in a different bucket (1 element in each bucket)
        - If unevenly distributed, two or more buckets would have multiple elements. Some buckets may not have any elements - max gap could occur betweeen left and right of the missing buckets
    2. Consequently we only need to store the min and max of each bucket
    """
    N, high, low = len(nums), max(nums), min(nums)
    bucket_size = math.ceil((high - low) / N)
    buckets: dict[int, tuple[int, int]] = {}

    # Edge case
    if bucket_size == 0:
        return 0

    # Group the numbers into buckets
    for n in nums:
        index = n // bucket_size
        min_, max_ = buckets.get(index, ((index + 1) * bucket_size, (index - 1) * bucket_size))
        min_, max_ = min(min_, n), max(max_, n)
        buckets[index] = (min_, max_)

    # Iterate through each and find the max gap
    curr, end = low // bucket_size, high // bucket_size
    max_gap = 0
    while curr < end:
        next_ = curr + 1
        while next_ not in buckets:
            next_ += 1
        curr, max_gap = next_, max(max_gap, buckets[next_][0] - buckets[curr][1])

    return max_gap

# Testing the solution
assert maximumGap([3,14,15,83,6,4,19,20,40]) == 43
assert maximumGap([3,6,9,1]) == 3
assert maximumGap([10]) == 0

LC Virtual - 5: Weekly Contest 6
https://leetcode.com/contest/leetcode-weekly-contest-6/

In [ ]:
# Q1: Sum of left leaves
def sumOfLeftLeaves(root: BinaryTreeNode) -> int:
    def backtrack(curr: BinaryTreeNode) -> tuple[int, int]:
        if not curr:
            return 0, -1
        else:
            left_sum, left_type = backtrack(curr.left)
            right_sum, right_type = backtrack(curr.right)
            curr_type, total = 0 if left_type == -1 and right_type == -1 else 1, 0
            if curr_type == 0:
                total += curr.val
            if left_type != -1:
                total += left_sum
            if right_type >= 1:
                total += right_sum
            result = total, curr_type
            return result

    return backtrack(root)[0] if root.left or root.right else 0

# Testing the solution
assert sumOfLeftLeaves(BinaryTreeNode.from_array([3,9,20,None,None,15,7])) == 24
assert sumOfLeftLeaves(BinaryTreeNode.from_array([1])) == 0

In [ ]:
# Q2: Convert to hexadecimal
def toHex(num: int) -> str:
    # If negative, find hex for its 2's compliment
    if num < 0:
        bit_length = 32
        num &= (2 << bit_length - 1) - 1

    # Use division method
    division_result: list[int] = []
    while num >= 16:
        num, rem = num // 16, num % 16
        division_result.append(rem)
    division_result.append(num)
    division_result.reverse()

    # Convert to hexadecimal string
    mapping = {i: str(i) for i in range(0, 10)} | {i: chr(ord('a') + i - 10) for i in range(10, 16)}
    return ''.join(map(lambda x: mapping[x], division_result))

# Testing the solution
assert toHex(26) == "1a"
assert toHex(-1) == "ffffffff"

In [ ]:
# Q3: Queue Reconstruction by Height
def reconstructQueue(people: list[list[int]]) -> list[list[int]]:
    N = len(people)

    # Sort to find the relative height ordering
    people.sort()

    # Iterate from shortest to tallest, keeping track of number of person with height equal to curr
    eq_front = []
    prev, eq_count = -1, 0
    for i in range(N):
        if people[i][0] == prev:
            eq_count += 1
        else:
            eq_count = 0
        prev = people[i][0]
        eq_front.append(eq_count)

    # Insert into queue in correct order
    result: list[list[int]] = []
    while eq_front:
        actual, (height, expected) = eq_front.pop(), people.pop()
        if expected == actual:
            result.append([height, expected])
        else:
            temp: list[list[int]] = []
            for i in range(expected - actual):
                temp.append(result.pop())
            result.append([height, expected])
            while temp:
                result.append(temp.pop())

    result.reverse()
    return result

# Testing the solution
assert reconstructQueue([[7,0],[4,4],[7,1],[5,0],[6,1],[5,2]]) == [[5,0],[7,0],[5,2],[6,1],[4,4],[7,1]]
assert reconstructQueue([[6,0],[5,0],[4,0],[3,2],[2,2],[1,4]]) == [[4,0],[5,0],[2,2],[3,2],[1,4],[6,0]]